In [1]:
# 화면 가로 확장 코드 (기본 width 50%)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

### 데이터 판다스로 입력, 컬럼 정리

In [2]:
import pandas as pd
ch1 = pd.read_csv('../../../channel_1.dat', names=['Unix', 'Watt'],header=None, delimiter=" ")
print('Module Import Done.')

Module Import Done.


In [3]:
ch1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21837636 entries, 0 to 21837635
Data columns (total 2 columns):
 #   Column  Dtype
---  ------  -----
 0   Unix    int64
 1   Watt    int64
dtypes: int64(2)
memory usage: 333.2 MB


In [4]:
from datetime import datetime
ch1_dated = pd.DataFrame(ch1,columns = ['Unix', 'Date', 'Watt'])

ch1_dated['Date'] = pd.to_datetime(ch1['Unix'], unit='s')
ch1_dated.info()
ch1_dated.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21837636 entries, 0 to 21837635
Data columns (total 3 columns):
 #   Column  Dtype         
---  ------  -----         
 0   Unix    int64         
 1   Date    datetime64[ns]
 2   Watt    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 499.8 MB


,Unix,Date,Watt
0,1352500095,2012-11-09 22:28:15,599
1,1352500101,2012-11-09 22:28:21,582
2,1352500107,2012-11-09 22:28:27,600
3,1352500113,2012-11-09 22:28:33,586
4,1352500120,2012-11-09 22:28:40,596
5,1352500132,2012-11-09 22:28:52,581
6,1352500138,2012-11-09 22:28:58,597
7,1352500144,2012-11-09 22:29:04,592
8,1352500150,2012-11-09 22:29:10,587
9,1352500156,2012-11-09 22:29:16,574


In [5]:
ch1_dated[431369:431389]

,Unix,Date,Watt
431369,1356998407,2013-01-01 00:00:07,500
431370,1356998413,2013-01-01 00:00:13,515
431371,1356998419,2013-01-01 00:00:19,522
431372,1356998426,2013-01-01 00:00:26,508
431373,1356998432,2013-01-01 00:00:32,502
431374,1356998438,2013-01-01 00:00:38,507
431375,1356998444,2013-01-01 00:00:44,508
431376,1356998450,2013-01-01 00:00:50,500
431377,1356998456,2013-01-01 00:00:56,517
431378,1356998462,2013-01-01 00:01:02,505


### 변수 및 구분 단계

### csv 파일 저장을 위한 폴더 생성

In [6]:
import os
try:
    os.mkdir('./New_Electirc_Data_CSV')
except FileExistsError:
    print('해당 폴더가 이미 존재합니다.\n폴더를 확인해주세요.')

### 데이터 변환 및 CSV 파일 저장 코드

### 중간 정보 확인창 출력 없이 시작, 종료 메세지만 출력

In [9]:
import csv
import time
import sys
month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

print("해당 전력 데이터는 2012년 11월 10일부터 2017년 4월 25일까지 온전한 데이터가 존재합니다.\n")
time.sleep(0.5)
print("데이터는 사용자가 선택한 기간 중 1년 단위로 처리됩니다.\n")
time.sleep(0.5)
global TARGET_YEAR_START 
TARGET_YEAR_START = int(input("목표로 하는 처리 기간의 시작 연도를 4자리 숫자로 입력해주세요. : \n"))
time.sleep(0.5)
global TARGET_YEAR_END
TARGET_YEAR_END = int(input("목표로 하는 처리 기간의 종료 연도를 4자리 숫자로 입력해주세요. : \n"))
time.sleep(0.5)

if(TARGET_YEAR_START<2012 or TARGET_YEAR_START>2017 or TARGET_YEAR_END>2017 or  TARGET_YEAR_END<2012):
    print("\---기간을 잘못 설정했습니다!---")
    print("---프로그램을 종료합니다.---")
    sys.exit()

if(TARGET_YEAR_START==2012 or TARGET_YEAR_END==2017):
    print("\n---해당 기간은 현재 서비스하지 않습니다!---")
    print("---프로그램을 종료합니다.---")
    sys.exit()
    
global HANDLE_MINUTE
HANDLE_MINUTE = int(input("데이터들은 기본적으로 1분 단위 최댓값으로 정리됩니다.\n추후 가공될 분 단위를 5분과 10분 중 골라 숫자만 입력해주세요. : \n"))
if(HANDLE_MINUTE!=5 and HANDLE_MINUTE!=10):
    print("\n---잘못된 시간 처리 단위를 선택했습니다!---")
    print("---프로그램을 종료합니다.---")
    sys.exit()
    
def value (data, num):
    list0 = data.loc[num].values.tolist()
    date = list0[1]
    YYYY = date.year
    MM = date.month
    DD = date.day
    hh = date.hour
    mm = date.minute
    watt = list0[2]
    return YYYY, MM, DD, hh, mm, watt

print(value(ch1_dated,431369)[0])

#i1=0
i1=431369
while(i1 < len(ch1_dated)):
    print("전체 과정 시작")
    YEAR0 = value(ch1_dated, i1)[0]
    #맨안쪽 1분의 안쪽 6초 단위에서 i1을 증가시켜줘야함.
    if(YEAR0>=TARGET_YEAR_START and YEAR0<=TARGET_YEAR_END):
        print("년 단위 시작")
        YEAR1 = value(ch1_dated, i1)[0]
        MONTH0 = value(ch1_dated, i1)[1]
        
        while(MONTH0<13):
            print("월 단위 시작")
            YEAR2 = value(ch1_dated, i1)[0]
            MONTH1 = value(ch1_dated, i1)[1]
            DAY0 = value(ch1_dated, i1)[2]
            
            while(DAY0<month[MONTH1]):
                print("하루 단위 시작")
                YEAR3 = value(ch1_dated, i1)[0]
                MONTH2 = value(ch1_dated, i1)[1]
                DAY1 = value(ch1_dated, i1)[2]
                HOUR0 = value(ch1_dated, i1)[3]
                
                while(HOUR0<24):
                    print("한시간 단위 시작")
                    YEAR4 = value(ch1_dated, i1)[0]
                    MONTH3 = value(ch1_dated, i1)[1]
                    DAY2 = value(ch1_dated, i1)[2]
                    HOUR1 = value(ch1_dated, i1)[3]
                    MINUTE0 = value(ch1_dated, i1)[4]
                    
                    MIN_LIST=[]
                    REAL_HOUUR=0
                    
                    while(MINUTE0<60):
                        YEAR5 = value(ch1_dated, i1)[0]
                        MONTH4 = value(ch1_dated, i1)[1]
                        DAY3 = value(ch1_dated, i1)[2]
                        HOUR2 = value(ch1_dated, i1)[3]
                        MINUTE1 = value(ch1_dated, i1)[4]
                        print("1분 단위 시작")
                        
                        SEC_60_LIST=[]
                        REAL_MINUUTE=0
                        while(True):
                            YEAR6 = value(ch1_dated, i1)[0]
                            MONTH5 = value(ch1_dated, i1)[1]
                            DAY4 = value(ch1_dated, i1)[2]
                            HOUR3 = value(ch1_dated, i1)[3]
                            MINUTE2 = value(ch1_dated, i1)[4]
                            
                            if MINUTE2==REAL_MINUTE:
                                SEC_60_LIST.append(value(ch1_dated, i1)[5])
                                print(MINUTE1, MINUTE2)
                                i1+=1
                                REAL_MINUUTE+=1
                            if MINUTE2!=REAL_MINUTE:#데이터상 특정분이 결손된 경우, 데이터의 분대는 원래 있어야할 분대보다 크다
                                SEC_60_LIST.append(0)
                                REAL_MINUUTE+=1
                            if MINUTE1!=MINUTE2:
                                i1-=1
                                print("* {0} 분대의 값들 >> {1}\n".format(MINUTE1, SEC_60_LIST))
                                break
                            if REAL_MINUTE==59:
                                ##############################어라 각 변수들을 숫자 붙혀서 구분할 필요가 없는거 아닌가?
                            else:
                                print("* Unknown Error!!")
                                sys.exit()
                        
                        
                        MIN_LIST.append(max( SEC_60_LIST))
                        if HOUR2!=HOUR3:
                            print("* {0} 시까지의 분별 최댓값 >> {1}\n".format(HOUR2, MIN_LIST))
                            break
                            # 현재 중간 널값에 대한 처리가 없음!!
                            # 지난번 코드처럼 순번을 카운트하는게 필요함. 절대적인 횟수 순번
        
    
    

        
    else:
        i1+=1
        pass

해당 전력 데이터는 2012년 11월 10일부터 2017년 4월 25일까지 온전한 데이터가 존재합니다.

데이터는 사용자가 선택한 기간 중 1년 단위로 처리됩니다.

목표로 하는 처리 기간의 시작 연도를 4자리 숫자로 입력해주세요. : 
2013
목표로 하는 처리 기간의 종료 연도를 4자리 숫자로 입력해주세요. : 
2013
데이터들은 기본적으로 1분 단위 최댓값으로 정리됩니다.
추후 가공될 분 단위를 5분과 10분 중 골라 숫자만 입력해주세요. : 
5
2013
전체 과정 시작
년 단위 시작
월 단위 시작
하루 단위 시작
한시간 단위 시작
1분 단위 시작
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
* 0 분대의 값들 >> [500, 515, 522, 508, 502, 507, 508, 500, 517, 505]

1분 단위 시작
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 2
* 1 분대의 값들 >> [505, 538, 551, 530, 558, 522, 514, 505, 511, 517, 519]

1분 단위 시작
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 3
* 2 분대의 값들 >> [519, 529, 512, 530, 513, 537, 513, 512, 508, 522, 525]

1분 단위 시작
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 4
* 3 분대의 값들 >> [525, 513, 514, 510, 500, 511, 513, 503, 525, 505]

1분 단위 시작
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 5
* 4 분대의 값들 >> [505, 510, 501, 521, 509, 511, 500, 517, 510, 539, 517]

1분 단위 시작
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 6
* 5 분대의 값들 >> [517, 516, 509, 551, 515, 518

11 11
11 12
* 11 분대의 값들 >> [284, 282, 277, 282, 280, 281, 285, 279, 283, 278, 285]

1분 단위 시작
12 12
12 12
12 12
12 12
12 12
12 12
12 12
12 12
12 12
12 12
12 13
* 12 분대의 값들 >> [285, 280, 278, 283, 278, 277, 278, 282, 276, 277, 283]

1분 단위 시작
13 13
13 13
13 13
13 13
13 13
13 13
13 13
13 13
13 13
13 13
13 14
* 13 분대의 값들 >> [283, 279, 281, 280, 281, 280, 284, 280, 284, 280, 285]

1분 단위 시작
14 14
14 14
14 14
14 14
14 14
14 14
14 14
14 14
14 14
14 15
* 14 분대의 값들 >> [285, 279, 284, 284, 283, 280, 286, 279, 280, 277]

1분 단위 시작
15 15
15 15
15 15
15 15
15 15
15 15
15 15
15 15
15 15
15 16
* 15 분대의 값들 >> [277, 282, 282, 285, 278, 284, 285, 281, 283, 279]

1분 단위 시작
16 16
16 16
16 16
16 16
16 16
16 16
16 16
16 16
16 16
16 16
16 17
* 16 분대의 값들 >> [279, 282, 279, 280, 282, 283, 283, 278, 283, 283, 278]

1분 단위 시작
17 17
17 17
17 17
17 17
17 17
17 17
17 17
17 17
17 17
17 17
17 18
* 17 분대의 값들 >> [278, 283, 282, 279, 281, 281, 285, 279, 279, 279, 282]

1분 단위 시작
18 18
18 18
18 18
18 18
18 18
18 18
18 18
18 18

24 25
* 24 분대의 값들 >> [183, 182, 181, 186, 186, 184, 184, 187, 183, 185, 191]

1분 단위 시작
25 25
25 25
25 25
25 25
25 25
25 25
25 25
25 26
* 25 분대의 값들 >> [191, 186, 185, 249, 248, 244, 244, 243]

1분 단위 시작
26 26
26 26
26 26
26 26
26 26
26 26
26 26
26 26
26 26
26 27
* 26 분대의 값들 >> [243, 242, 240, 240, 239, 244, 240, 243, 240, 244]

1분 단위 시작
27 27
27 27
27 27
27 27
27 27
27 27
27 27
27 27
27 27
27 27
27 28
* 27 분대의 값들 >> [244, 243, 242, 241, 238, 248, 240, 242, 242, 242, 240]

1분 단위 시작
28 28
28 28
28 28
28 28
28 28
28 28
28 28
28 28
28 28
28 28
28 29
* 28 분대의 값들 >> [240, 238, 243, 241, 240, 246, 240, 244, 239, 243, 244]

1분 단위 시작
29 29
29 29
29 29
29 29
29 29
29 29
29 29
29 29
29 29
29 29
29 30
* 29 분대의 값들 >> [244, 242, 239, 240, 240, 241, 240, 243, 250, 246, 244]

1분 단위 시작
30 30
30 30
30 30
30 30
30 30
30 30
30 30
30 30
30 30
30 31
* 30 분대의 값들 >> [244, 238, 246, 240, 241, 236, 236, 240, 240, 240]

1분 단위 시작
31 31
31 31
31 31
31 31
31 31
31 31
31 31
31 31
31 32
* 31 분대의 값들 >> [240, 240, 239, 2

37 37
37 38
* 37 분대의 값들 >> [239, 239, 241, 239, 239, 240, 239, 240, 239]

1분 단위 시작
38 38
38 38
38 38
38 38
38 38
38 38
38 38
38 38
38 38
38 38
38 39
* 38 분대의 값들 >> [239, 238, 241, 241, 242, 239, 239, 241, 239, 242, 242]

1분 단위 시작
39 39
39 39
39 39
39 39
39 39
39 39
39 39
39 39
39 39
39 39
39 40
* 39 분대의 값들 >> [242, 243, 240, 241, 244, 238, 241, 240, 242, 243, 243]

1분 단위 시작
40 40
40 40
40 40
40 40
40 40
40 40
40 40
40 40
40 40
40 40
40 41
* 40 분대의 값들 >> [243, 243, 239, 241, 243, 244, 242, 238, 243, 239, 241]

1분 단위 시작
41 41
41 41
41 41
41 41
41 41
41 41
41 41
41 41
41 41
41 42
* 41 분대의 값들 >> [241, 242, 242, 240, 240, 242, 242, 244, 240, 240]

1분 단위 시작
42 42
42 42
42 42
42 42
42 42
42 42
42 42
42 42
42 42
42 43
* 42 분대의 값들 >> [240, 239, 242, 240, 243, 242, 239, 241, 243, 240]

1분 단위 시작
43 43
43 43
43 43
43 43
43 43
43 43
43 43
43 43
43 43
43 43
43 44
* 43 분대의 값들 >> [240, 243, 240, 242, 240, 239, 241, 242, 240, 242, 240]

1분 단위 시작
44 44
44 44
44 44
44 44
44 44
44 44
44 44
44 44
44 44
44 

1분 단위 시작
51 51
51 51
51 51
51 51
51 51
51 51
51 51
51 51
51 51
51 51
51 52
* 51 분대의 값들 >> [181, 182, 178, 179, 182, 180, 181, 182, 178, 179, 183]

1분 단위 시작
52 52
52 52
52 52
52 52
52 52
52 52
52 52
52 52
52 52
52 52
52 53
* 52 분대의 값들 >> [183, 178, 181, 181, 179, 183, 178, 180, 179, 179, 186]

1분 단위 시작
53 53
53 53
53 53
53 53
53 53
53 53
53 53
53 53
53 54
* 53 분대의 값들 >> [186, 179, 178, 179, 179, 179, 181, 178, 181]

1분 단위 시작
54 54
54 54
54 54
54 54
54 54
54 54
54 54
54 54
54 54
54 55
* 54 분대의 값들 >> [181, 178, 179, 182, 179, 181, 178, 181, 181, 181]

1분 단위 시작
55 55
55 55
55 55
55 55
55 55
55 55
55 55
55 55
55 55
55 55
55 56
* 55 분대의 값들 >> [181, 178, 182, 181, 181, 181, 178, 181, 179, 181, 179]

1분 단위 시작
56 56
56 56
56 56
56 56
56 56
56 56
56 56
56 56
56 56
56 56
56 57
* 56 분대의 값들 >> [179, 181, 178, 180, 180, 181, 177, 178, 180, 181, 186]

1분 단위 시작
57 57
57 57
57 57
57 57
57 57
57 57
57 57
57 57
57 57
57 57
57 58
* 57 분대의 값들 >> [186, 179, 180, 178, 179, 180, 179, 181, 181, 179, 181]

1분 단

2 2
2 2
2 3
* 2 분대의 값들 >> [179, 182, 180, 179, 183, 182, 181, 183, 180, 180, 181]

1분 단위 시작
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 3
3 4
* 3 분대의 값들 >> [181, 182, 180, 180, 180, 180, 180, 182, 180, 182, 186]

1분 단위 시작
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 5
* 4 분대의 값들 >> [186, 182, 182, 180, 182, 183, 182, 183, 180, 179, 180]

1분 단위 시작
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 5
5 6
* 5 분대의 값들 >> [180, 179, 181, 179, 179, 181, 181, 179, 181, 182]

1분 단위 시작
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 6
6 7
* 6 분대의 값들 >> [182, 178, 181, 179, 182, 181, 181, 178, 180, 180, 180]

1분 단위 시작
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 7
7 8
* 7 분대의 값들 >> [180, 181, 184, 182, 180, 182, 181, 182, 183, 182, 184]

1분 단위 시작
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 9
* 8 분대의 값들 >> [184, 180, 183, 180, 180, 179, 180, 182, 181, 181]

1분 단위 시작
9 9
9 9
9 9
9 9
9 9
9 9
9 9
9 9
9 9
9 10
* 9 분대의 값들 >> [181, 182, 182, 183, 182, 179, 183, 183, 179, 182]

1분 단위 시작
10 10
10 10
10 10
10 10
10 10
10 10
10 10
10 10
10 10
10 10
10 11


14 15
* 14 분대의 값들 >> [183, 183, 182, 185, 184, 184, 218, 192, 190, 194, 186]

1분 단위 시작
15 15
15 15
15 15
15 15
15 15
15 15
15 15
15 15
15 15
15 16
* 15 분대의 값들 >> [186, 194, 190, 248, 240, 240, 237, 236, 235, 229]

1분 단위 시작
16 16
16 16
16 16
16 16
16 16
16 16
16 16
16 16
16 16
16 16
16 17
* 16 분대의 값들 >> [229, 231, 232, 229, 228, 230, 231, 227, 228, 231, 229]

1분 단위 시작
17 17
17 17
17 17
17 17
17 17
17 17
17 17
17 17
17 18
* 17 분대의 값들 >> [229, 231, 231, 228, 230, 227, 230, 230, 229]

1분 단위 시작
18 18
18 18
18 18
18 18
18 18
18 18
18 18
18 18
18 18
18 19
* 18 분대의 값들 >> [229, 229, 229, 230, 226, 252, 247, 246, 245, 260]

1분 단위 시작
19 19
19 19
19 19
19 19
19 19
19 19
19 19
19 19
19 19
19 19
19 20
* 19 분대의 값들 >> [260, 257, 258, 257, 255, 259, 256, 256, 255, 256, 258]

1분 단위 시작
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 21
* 20 분대의 값들 >> [258, 260, 257, 261, 259, 260, 261, 262, 260, 264, 261]

1분 단위 시작
21 21
21 21
21 21
21 21
21 21
21 21
21 21
21 21
21 21
21 21
21 22
* 21 분대의 

25 26
* 25 분대의 값들 >> [273, 269, 266, 268, 269, 268, 264, 266, 271, 264, 255]

1분 단위 시작
26 26
26 26
26 26
26 26
26 26
26 26
26 26
26 26
26 26
26 26
26 27
* 26 분대의 값들 >> [255, 257, 259, 258, 250, 252, 255, 253, 247, 250, 257]

1분 단위 시작
27 27
27 27
27 27
27 27
27 27
27 27
27 27
27 27
27 27
27 27
27 28
* 27 분대의 값들 >> [257, 251, 254, 257, 248, 249, 250, 250, 248, 247, 251]

1분 단위 시작
28 28
28 28
28 28
28 28
28 28
28 28
28 28
28 28
28 28
28 29
* 28 분대의 값들 >> [251, 253, 247, 253, 249, 247, 247, 251, 250, 252]

1분 단위 시작
29 29
29 29
29 29
29 29
29 29
29 29
29 29
29 29
29 29
29 29
29 30
* 29 분대의 값들 >> [252, 250, 255, 248, 254, 259, 248, 251, 250, 249, 253]

1분 단위 시작
30 30
30 30
30 30
30 30
30 30
30 30
30 30
30 30
30 30
30 30
30 31
* 30 분대의 값들 >> [253, 250, 252, 250, 248, 258, 250, 252, 284, 277, 273]

1분 단위 시작
31 31
31 31
31 31
31 31
31 31
31 31
31 31
31 31
31 31
31 32
* 31 분대의 값들 >> [273, 274, 269, 268, 268, 269, 267, 264, 265, 270]

1분 단위 시작
32 32
32 32
32 32
32 32
32 32
32 32
32 32
32 33
* 32 

KeyboardInterrupt: 

### 아래가 원본

In [ ]:
import csv
import time

month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

# 1. 
i = 431369#431369가 맞음
print('Running!', '\n'*4)


# 2. 
while(i < 20284837):
    try:
        list1 = ch1_date.loc[i].values.tolist()
        date = list1[1]
        y1 = date.year
        M1 = date.month
        d1 = date.day
        H1 = date.hour 
        n = 0 

        # 3. 
        while(H1 < 24):
            H0 = 0 

            # 3'. 
            while(H0 < 24):
                watt_list=[]
                h = date.hour

                # 3.1. 
                if(H1 == H0):

                    # 3.1'. 
                    while(h < h+1):
                        n=n+1
                        list2 = ch1_date.loc[i].values.tolist()
                        date = list2[1]
                        watt = list2[2]
                        y = date.year
                        M = date.month
                        d = date.day
                        h = date.hour
                        m = date.minute
                        s = date.second

                        j=h-H1

                        # 3.1'.1. 
                        if(j==0):
                            watt_list.append(watt)
                            i = i+1
                            """
                            여기서 1시간 내의 과정이 이뤄지니 그안에서 5분이든 10분이든 처리를 해줘야 함.
                            """

                        # 3.1'.2.
                        elif (j==1):
                            max_watt = max(watt_list)

                            hourly_date2 = '{0}년 {1}월 {2}일 - {3}시'.format(y, M, d, H1)
                            hourly_date5 = '{0} {1} {2}'.format(y, M, d) 
                            hourly_list = []
                            hourly_list.append(hourly_date2)
                            """
                            밑에 있는 hourly_list는 1시간 단위로 max_watt의 값을 가져오는 것이니 여기서도 주기에 맞게 처리를 해줘야함
                            
                            밑에 있는 max_watt 등장 부분 모두 마찬가지.
                            """
                            hourly_list.append(max_watt)

                            with open("./Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(hourly_list)       

                            H1 = H1+1 
                            H0 = H0+1
                            break

                        # 3.1'.3. 
                        elif(d1+1==d and h ==0 and H1 ==23):
                            d=d-1
                            h=h+24

                            max_watt = max(watt_list)

                            hourly_date3 = '{0}년 {1}월 {2}일 - {3}시'.format(y, M, d, H1)
                            hourly_list = []
                            hourly_list.append(hourly_date3)
                            hourly_list.append(max_watt)

                            with open("./Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(hourly_list)       

                            H1 = H1+1 
                            H0 = H0+1                   
                            break

                        # 3.1'.4.
                        elif (n>600):
                            list(hourly_date5)
                            listed = hourly_date5.split()
                            y4 = listed[0]
                            M4 = listed[1]
                            d4 = listed[2]

                            max_watt = max(watt_list)

                            hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, d4, H1)
                            hourly_list = []
                            hourly_list.append(hourly_date)
                            hourly_list.append(max_watt)

                            with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(hourly_list)       

                            # 3.1'.4.1. 
                            if(d1==d):
                                jc=j

                                while(jc>1):
                                    h=h+1
                                    hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, d4, h-j)
                                    hourly_list = []
                                    hourly_list.append(hourly_date)
                                    hourly_list.append('Null')

                                    with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(hourly_list)  

                                    jc=jc-1
                                    H1 = H1+1
                                    H0 = H0+1

                                H1 = H1+1
                                H0 = H0+1

                            # 3.1'.4.2.
                            if(d1!=d):

                                while(H1<23):
                                    hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, d4, H1+1)
                                    hourly_list = []
                                    hourly_list.append(hourly_date)
                                    hourly_list.append('Null')

                                    with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(hourly_list)  

                                    H1 = H1+1
                                    H0 = H0+1

                                H1 = H1+1
                                H0 = H0+1

                                # 3.1'.4.2.1. 
                                if(int(d4)+1 != d):
                                    D = int(d4)+1 

                                    # 3.1'.4.2.1.1. 
                                    if(int(M) == int(M4)):
                                        while(D<d):
                                            for x in range (0,24):
                                                hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, D, x)
                                                hourly_list = []
                                                hourly_list.append(hourly_date)
                                                hourly_list.append('Null')

                                                with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                                    writer = csv.writer(file)
                                                    writer.writerow(hourly_list)  

                                            D=D+1

                                    # 3.1'.4.2.1.2.
                                    if(int(M) != int(M4)):
                                        while(D<=month[int(M4)]):
                                            for x in range (0,24):
                                                hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M4, D, x)
                                                hourly_list = []
                                                hourly_list.append(hourly_date)
                                                hourly_list.append('Null')

                                                with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M4)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                                    writer = csv.writer(file)
                                                    writer.writerow(hourly_list)  

                                            D=D+1

                                        D2 = 1
                                        while(D2 < d):
                                            for x in range (0,24):
                                                hourly_date = '{0}년 {1}월 {2}일 - {3}시'.format(y4, M, D2, x)
                                                hourly_list = []
                                                hourly_list.append(hourly_date)
                                                hourly_list.append('Null')

                                                with open("./Electirc_Data_CSV/"+str(y4)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                                                    writer = csv.writer(file)
                                                    writer.writerow(hourly_list)  

                                            D2 = D2+1

                            break


                # 3.2. 
                if(H1 != H0):
                    list3 = ch1_date.loc[i].values.tolist()
                    date = list3[1]
                    watt = list3[2]
                    y = date.year
                    M = date.month
                    d = date.day
                    h = date.hour
                    m = date.minute
                    s = date.second

                    j=h-H1           

                    hourly_date3121 = '{0}년 {1}월 {2}일 - {3}시'.format(y, M, d, H0)
                    hourly_list3121 = []
                    hourly_list3121.append(hourly_date3121)
                    hourly_list3121.append('Null')

                    with open("./Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch1_Electric_Data.csv", 'a', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow(hourly_list3121)       

                    h = h+1
                    H0 = H0+1

                    if (H0 ==23 and h!=24 and H1!=23):
                                break

                    if(h==24 and H0==23 and H1==23):
                        hourly_date3121 = '{0}년 {1}월 {2}일 - 23시'.format(y, M, d)
                        hourly_list3121 = []
                        hourly_list3121.append(hourly_date3121)
                        hourly_list3121.append('Null')

                        with open("./Ch"+str(Total_num)+"_Electirc_Data_CSV/"+str(y)+'_'+str(M)+'_'+"House1_Ch"+str(Total_num)+"_Electric_Data.csv", 'a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(hourly_list3121)

            break
    except:
        pass
    
print('Job Done!!')
time.sleep(5)